# Parsing (JUNOS)

One of the features ntc_rosetta supports is parsing native configuration and turning into data modelled after YANG models. For that purpose ntc_rosetta leverages [yangify](https://github.com/networktocode/yangify) and builds on top of it to make it more consumable.

ntc_rosetta introduces the concept of "drivers". Drivers are objects that implements the parsing and translation of a given YANG model for a particular NOS. For instance, if you wanted to parse IOS configuration and convert it into data that follows the openconfig model you would load the corresponding driver like this:

In [1]:
from ntc_rosetta import get_driver

junos = get_driver("junos", "openconfig")
junos_driver = junos()

The same processor can also translate the given model to native configuration.

Now, let's see how we can use this driver to parse IOS configuration and turn it into an Openconfig model. First, let's load some IOS configuration:

In [2]:
with open("data/junos/dev_conf.xml", "r") as f:
    config = f.read()

In [3]:
print(config)

<configuration>
        <system>
            <host-name>junos-host</host-name>
            <name-server>
                <name>8.8.8.8</name>
            </name-server>
            <name-server>
                <name>8.8.4.4</name>
            </name-server>
            <name-server>
                <name>2606:4700:4700::1111</name>
            </name-server>
            <name-server>
                <name>2606:4700:4700::1001</name>
            </name-server>
            <domain-name>packet.net</domain-name>
            <tacplus-server>
                <name>192.168.0.42</name>
                <secret>$9$kq5z3nCuBE/CLNb2GU/Ctp0IEcy</secret>
            </tacplus-server>
            <services>
                <ssh>
                    <protocol-version>v2</protocol-version>
                </ssh>
            </services>
            <login>
                <user>
                    <name>user</name>
                    <class>super-user</class>
                    <authentication>
    

Once the configuration is loaded, you need to parse it. The parser has some conventions you have to be aware of, for instance, when parsing configuration, it's going to expect you pass a `native` argument with a dictionary where the key `dev_conf` is the native configuration:

In [4]:
parsed = junos_driver.parse(native={"dev_conf": config})

That's literally all you have to do parse the native configuration and turn it into structured data. We can check the result by dumping the `parsed.raw_value()`:

In [5]:
import json
print(json.dumps(parsed.raw_value(), sort_keys=True, indent=4))

{
    "openconfig-interfaces:interfaces": {
        "interface": [
            {
                "config": {
                    "enabled": true,
                    "name": "xe-0/0/1",
                    "type": "iana-if-type:ethernetCsmacd"
                },
                "name": "xe-0/0/1",
                "openconfig-if-ethernet:ethernet": {
                    "openconfig-vlan:switched-vlan": {
                        "config": {
                            "access-vlan": 10,
                            "interface-mode": "ACCESS"
                        }
                    }
                },
                "subinterfaces": {
                    "subinterface": [
                        {
                            "config": {
                                "index": 0
                            },
                            "index": 0
                        }
                    ]
                }
            },
            {
                "config": {
             

ntc_rosetta, also let's you parse some parts of the model, however, you need to be aware that might break the validation of the object:

In [6]:
from yangson.exceptions import SemanticError
try:
    parsed_vlans = junos_driver.parse(
        native={"dev_conf": config},
        include=[
            "/openconfig-network-instance:network-instances/network-instance/vlans",        
        ]
    )
except SemanticError as e:
    print(f"error: {e}")

error: [/openconfig-network-instance:network-instances/network-instance/0/name] instance-required


You can workaround this in two ways:
1. By disabling the validation of the object
2. By parsing all the necessary elements to make the object compliant.

You can disable the validation of the object by passing `validate=False`:

In [7]:
parsed_vlans = junos_driver.parse(
    native={"dev_conf": config},
    validate=False,
    include=[
        "/openconfig-network-instance:network-instances/network-instance/vlans",        
    ]
)
print(json.dumps(parsed_vlans.raw_value(), sort_keys=True, indent=4))

{
    "openconfig-network-instance:network-instances": {
        "network-instance": [
            {
                "name": "default",
                "vlans": {
                    "vlan": [
                        {
                            "config": {
                                "name": "default",
                                "status": "ACTIVE",
                                "vlan-id": 1
                            },
                            "vlan-id": 1
                        },
                        {
                            "config": {
                                "name": "prod",
                                "status": "ACTIVE",
                                "vlan-id": 20
                            },
                            "vlan-id": 20
                        },
                        {
                            "config": {
                                "status": "SUSPENDED",
                                "vlan-id": 10
               

And you can make sure your object is valid by passing the list of elements that are needed to make the object compliant:

In [8]:
parsed_vlans = junos_driver.parse(
    native={"dev_conf": config},
    include=[
        "/openconfig-network-instance:network-instances/network-instance/name",
        "/openconfig-network-instance:network-instances/network-instance/config",
        "/openconfig-network-instance:network-instances/network-instance/vlans",        
    ]
)
print(json.dumps(parsed_vlans.raw_value(), sort_keys=True, indent=4))

{
    "openconfig-network-instance:network-instances": {
        "network-instance": [
            {
                "config": {
                    "name": "default"
                },
                "name": "default",
                "vlans": {
                    "vlan": [
                        {
                            "config": {
                                "name": "default",
                                "status": "ACTIVE",
                                "vlan-id": 1
                            },
                            "vlan-id": 1
                        },
                        {
                            "config": {
                                "name": "prod",
                                "status": "ACTIVE",
                                "vlan-id": 20
                            },
                            "vlan-id": 20
                        },
                        {
                            "config": {
                               